# Supervised graph classification with Deep Graph CNN

<table><tr><td>Run the latest release of this notebook:</td><td><a href="https://mybinder.org/v2/gh/stellargraph/stellargraph/master?urlpath=lab/tree/demos/graph-classification/dgcnn-graph-classification.ipynb" alt="Open In Binder" target="_parent"><img src="https://mybinder.org/badge_logo.svg"/></a></td><td><a href="https://colab.research.google.com/github/stellargraph/stellargraph/blob/master/demos/graph-classification/dgcnn-graph-classification.ipynb" alt="Open In Colab" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg"/></a></td></tr></table>

In [1]:
# install StellarGraph if running on Google Colab
import sys
#if 'google.colab' in sys.modules:
#!pip install stellargraph[demos]==0.7.3
!pip install git+https://github.com/VenkateshwaranB/stellargraph.git

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

import pandas as pd


  Cloning https://github.com/VenkateshwaranB/stellargraph.git to /tmp/pip-req-build-i3vz6rqv
  Running command git clone --filter=blob:none --quiet https://github.com/VenkateshwaranB/stellargraph.git /tmp/pip-req-build-i3vz6rqv
  Resolved https://github.com/VenkateshwaranB/stellargraph.git to commit efa1f847109a4ba490e7a5105646a20ee09a3243
  Preparing metadata (setup.py) ... done
  Created wheel for stellargraph: filename=stellargraph-1.3.0b0-py3-none-any.whl size=431845 sha256=ef0454b8bd9a1e7f9b925efeb641c58448f7d073e0d0f7efc32957f23cb24c9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-rcmqv8we/wheels/f3/06/0f/089f69af27d308a1830638f855b6c5755311d8ffc451de9980
Successfully built stellargraph
Mounted at /content/drive


In [2]:
# verify that we're using the correct version of StellarGraph for this notebook
import stellargraph as sg

try:
    sg.utils.validate_notebook_version("1.1.0")
except AttributeError:
    raise ValueError(
        f"This notebook requires StellarGraph version 1.1.0, but a different version {sg.__version__} is installed.  Please see <https://github.com/stellargraph/stellargraph/issues/1172>."
    ) from None

<div class="alert alert-block alert-danger">This notebook is designed for an older StellarGraph version 1.1.0 and may not function correctly with the newer installed version 1.3.0b. Please see: <a href="https://github.com/stellargraph/stellargraph/issues/1172">https://github.com/stellargraph/stellargraph/issues/1172</a>.</div>

<ipython-input-2-345c07da7c9c>:5: DeprecationWarning: This notebook is designed for an older StellarGraph version 1.1.0 and may not function correctly with the newer installed version 1.3.0b. Please see: <https://github.com/stellargraph/stellargraph/issues/1172>.
  sg.utils.validate_notebook_version("1.1.0")


In [3]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import json
import math
import os
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet201
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
from pathlib import Path # help manage f
import gc
from functools import partial
from sklearn import metrics
import itertools
import seaborn as sns
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array

import pandas as pd
import os

import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

LUSC

In [4]:
from pathlib import Path
data = []

path22 =Path('/content/drive/My Drive/Datasets/Graph/Nodes/LUSC/')

data= pd.DataFrame({'h5_path': list(path22.glob('*'))})
data['file_id'] = data['h5_path'].map(lambda x: x.stem)

'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(len(data))
print (data.head(5)) # show the first 5 lines
#data.to_csv( '/home/robert/Downloads/Datasets/Node_features_lusc.csv')
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")

442
                                             h5_path       file_id
0  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.60.2714
1  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.58.8392
2  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.96.8169
3  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.94.A5I4
4  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.56.7221


In [5]:
from pathlib import Path
df = []

path =Path('/content/drive/My Drive/Datasets/Graph/Test4/')

df = pd.DataFrame({'h5_path': list(path.glob('*'))})
df['file_id'] = df['h5_path'].map(lambda x: x.stem)

'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(len(df))
print (df.head(5)) # show the first 5 lines
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")

273
                                             h5_path  file_id
0  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.60
1  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.58
2  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.96
3  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.56
4  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.77


In [6]:
from pathlib import Path
df17 = []

path17 =Path('/content/drive/My Drive/Datasets/Graph/Test4/')

df17 = pd.DataFrame({'h5_path': list(path17.glob('*'))})
#df7['f_id'] = 0

x = len(df17)
print(x)

for i in range(0, x): # Create an output directory
  #print("Slide", i, "of" ,m, "processing")

  im_path6 = df17.iloc[i]['h5_path'] # just take the first row
    #img = open_slide("/content/drive/My Drive/Datasets/Radiomics/new_00232.svs")
  #print(im_path6)

  fold_name = str(im_path6).split("/")[7]
  #print(fold_name)

  df17.loc[i,'f_id'] = fold_name



'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(df17.head())

print (len(df17)) # show the first 5 lines

out = data.loc[data.file_id.isin(df17.f_id)]
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")
print(len(out))


273
                                             h5_path          f_id
0  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.60.2714
1  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.58.8392
2  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.96.8169
3  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.56.7221
4  /content/drive/My Drive/Datasets/Graph/Test4/T...  TCGA.77.8133
273
273


LUAD

In [7]:
from pathlib import Path
data1 = []

path23=Path('/content/drive/My Drive/Datasets/Graph/Nodes/LUAD/')

data1= pd.DataFrame({'h5_path': list(path23.glob('*'))})
data1['file_id'] = data1['h5_path'].map(lambda x: x.stem)

'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(len(data1))
print (data1.head(5)) # show the first 5 lines
#data.to_csv( '/home/robert/Downloads/Datasets/Node_features_lusc.csv')
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")

418
                                             h5_path       file_id
0  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.97.A4M7
1  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.55.8507
2  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.49.4501
3  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.86.A4P8
4  /content/drive/My Drive/Datasets/Graph/Nodes/L...  TCGA.78.7153


In [8]:
from pathlib import Path
df = []

path =Path('/content/drive/My Drive/Datasets/Graph/Test2/')

df = pd.DataFrame({'h5_path': list(path.glob('*'))})
df['file_id'] = df['h5_path'].map(lambda x: x.stem)

'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(len(df))
print (df.head(5)) # show the first 5 lines
#df.to_csv( '/home/robert/Downloads/Datasets/Node_features_lusc.csv')
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")

201
                                             h5_path  file_id
0  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.55
1  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.49
2  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.86
3  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.78
4  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.44


In [9]:
from pathlib import Path
df27 = []

path27 =Path('/content/drive/My Drive/Datasets/Graph/Test2/')

df27 = pd.DataFrame({'h5_path': list(path27.glob('*'))})
#df7['f_id'] = 0

y= len(df27)
print(y)

for i in range(0, y): # Create an output directory
  #print("Slide", i, "of" ,m, "processing")

  im_path66 = df27.iloc[i]['h5_path'] # just take the first row
    #img = open_slide("/content/drive/My Drive/Datasets/Radiomics/new_00232.svs")
  #print(im_path6)

  fold_name1 = str(im_path66).split("/")[7]
  #print(fold_name)

  df27.loc[i,'f_id'] = fold_name1



'''
all_df['file_id'] = all_df['h5_path'].map(lambda x: x.stem)
all_df['training_group'] = all_df['h5_path'].map(lambda x: x.parent.stem)
all_df['scan_id'] = all_df['file_id'].map(lambda x: (x.split('.')[-2]))
all_df.set_index('scan_id', inplace = True)
'''
print(df27.head())

print (len(df27)) # show the first 5 lines

out1 = data1.loc[data1.file_id.isin(df27.f_id)]
#df.to_csv("/content/drive/My Drive/Datasets/Graph/lst.csv")
print(len(out1))

201
                                             h5_path          f_id
0  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.55.8507
1  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.49.4501
2  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.86.A4P8
3  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.78.7153
4  /content/drive/My Drive/Datasets/Graph/Test2/T...  TCGA.44.6775
201
201


In [10]:
m = len(out)
print(m)
graph_p = list()
#datanew = []
#data1 = pd.DataFrame(datanew)


for i in range(0, m): # Create an output directory
  #print("Slide", i, "of" ,m, "processing")

  im_path2 =[]
  fold_name = []
  edge_connections = []
  listOfFile = []
  path1 =[]
  node_data =[]
  graph1 =[]




  i#m_path2 = df.iloc[i]['h5_path'] # just take the first row
  fold_name = out.iloc[i]['file_id']

    #img = open_slide("/content/drive/My Drive/Datasets/Radiomics/new_00232.svs")
  #print(im_path2)

  #fold_name = df.iloc[i]['file_id']
  print(fold_name)
  #directory = "/content/drive/My Drive/Datasets/Tiles/LUSC1/TCGA.6A.AB49/"


  edge_path = '/content/drive/My Drive/Datasets/Graph/Nodes/LUSC/' + "/" + fold_name + ".csv"
  #edge_connections = pd.read_csv('/content/drive/My Drive/Datasets/Graph/Nodes/LUSC' + "/" + fold_name + ".csv", index_col='Unnamed: 0')
  edge_connections = pd.read_csv(edge_path, index_col='Unnamed: 0')
  #print(tile_connections.head())

  edge_connections = edge_connections.astype(int)
  print(edge_connections.head())

  if fold_name == 'TCGA.43.2576':
      continue

  if fold_name == 'TCGA.NC.A5HO':
      continue

  if fold_name == 'TCGA.85.7699':
      continue

  if fold_name == 'TCGA.96.A4JK':
      continue

  '''
  path1 =Path('/content/drive/My Drive/Datasets/Graph/Test/'+ fold_name)
  #path2 = str(path1) + "/" + str(fold_name)
  #path2 = path1 + "/" + fold_name
  newpath = str(path1.glob('*'))
  print(newpath)
  '''
  #if fold_name=='TCGA.94.A5I4':
    #continue

  listOfFile = os.listdir('/content/drive/My Drive/Datasets/Graph/Test4/'+ fold_name)
  #allFiles = list()
  print(listOfFile)

  base = os.path.splitext(listOfFile[0])[0]
  if int(base) > 2000:
    continue

  path1 = '/content/drive/My Drive/Datasets/Graph/Test4/'+ fold_name + "/" + listOfFile[0]


  #node_data = pd.read_csv('/content/drive/My Drive/Datasets/Graph/,index_col = 'Unnamed: 0')
  node_data = pd.read_csv(path1,index_col = 'Unnamed: 0')
  node_data = node_data.set_index('file_id')

  node_data = node_data.drop(['path'],axis=1)


  print(node_data.head())



  '''
  if i==0:
    graph_p = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
    print(graph_p.info())
  else:
    graph_p[i] = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
    print(graph_p[i].info())
  '''

  graph1 = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
  print(graph1.info())


  graph_p.append(graph1)

  #if i==1:
    #break

print(graph_p[20].info())

print(len(graph_p))


Streaming output truncated to the last 5000 lines.
422       2.678765  -4.576915  -6.146537  3.896041  0.530215  0.375805   
423       5.114021  -4.300910  -2.732175  2.733376  3.116376 -0.262481   
424       3.714067  -6.099145  -7.605016  1.887104 -0.365443 -0.105531   

                7         8         9        10  ...       247       248  \
file_id                                          ...                       
420     -0.451710  0.494409 -3.985054 -2.975122  ... -1.536734  9.310557   
421     -2.012125  0.132470 -3.407932  1.533319  ...  3.771031  8.178439   
422      1.245113 -3.262832 -2.506526 -1.442622  ...  4.606547  6.730709   
423     -2.830630 -2.605961 -1.522210 -0.936866  ... -2.602915  7.647193   
424      2.073065 -6.893185 -3.473060 -0.056005  ...  4.390223  8.850392   

              249        250       251       252       253       254  \
file_id                                                                
420     -6.271936  13.326149  3.845367 -4.776793 

In [11]:
q = len(out1)
print(q)
graph2_p = list()
#datanew = []
#data1 = pd.DataFrame(datanew)


for i in range(0, q): # Create an output directory
  #print("Slide", i, "of" ,m, "processing")

  edge_path =[]
  fold_name = []
  edge_connections = []
  listOfFile = []
  path1 =[]
  node_data =[]
  graph2 =[]




  i#m_path2 = df.iloc[i]['h5_path'] # just take the first row
  fold_name = out1.iloc[i]['file_id']

    #img = open_slide("/content/drive/My Drive/Datasets/Radiomics/new_00232.svs")
  #print(im_path2)

  #fold_name = df.iloc[i]['file_id']
  print(fold_name)
  #directory = "/content/drive/My Drive/Datasets/Tiles/LUSC1/TCGA.6A.AB49/"


  edge_path = '/content/drive/My Drive/Datasets/Graph/Nodes/LUAD/' + "/" + fold_name + ".csv"
  #edge_connections = pd.read_csv('/content/drive/My Drive/Datasets/Graph/Nodes/LUSC' + "/" + fold_name + ".csv", index_col='Unnamed: 0')
  edge_connections = pd.read_csv(edge_path, index_col='Unnamed: 0')
  #print(tile_connections.head())

  edge_connections = edge_connections.astype(int)
  print(edge_connections.head())

  if fold_name == 'TCGA.44.4112':
          continue

  if fold_name == 'TCGA.49.6745':
      continue

  if fold_name == 'TCGA.75.6212':
      continue

  if fold_name == 'TCGA.44.5645':
      continue
    #TCGA.96.A4JK

  if fold_name == 'TCGA.44.6147':
      continue

  if fold_name == 'TCGA.44.6775':
      continue

  if fold_name == 'TCGA.MP.A4T2':
      continue

  if fold_name == 'TCGA.86.8585':
      continue

  '''
  path1 =Path('/content/drive/My Drive/Datasets/Graph/Test/'+ fold_name)
  #path2 = str(path1) + "/" + str(fold_name)
  #path2 = path1 + "/" + fold_name
  newpath = str(path1.glob('*'))
  print(newpath)
  '''
  #if fold_name=='TCGA.94.A5I4':
    #continue

  listOfFile = os.listdir('/content/drive/My Drive/Datasets/Graph/Test2/'+ fold_name)
  #allFiles = list()
  print(listOfFile)

  base = os.path.splitext(listOfFile[0])[0]
  if int(base) > 2000:
    continue

  path1 = '/content/drive/My Drive/Datasets/Graph/Test2/'+ fold_name + "/" + listOfFile[0]


  #node_data = pd.read_csv('/content/drive/My Drive/Datasets/Graph/,index_col = 'Unnamed: 0')
  node_data = pd.read_csv(path1,index_col = 'Unnamed: 0')
  node_data = node_data.set_index('file_id')

  node_data = node_data.drop(['path'],axis=1)


  print(node_data.head())



  '''
  if i==0:
    graph_p = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
    print(graph_p.info())
  else:
    graph_p[i] = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
    print(graph_p[i].info())
  '''

  graph2 = StellarGraph( node_data, edges=edge_connections, source_column="0", target_column="1" )
  print(graph2.info())


  graph2_p.append(graph2)

  #if i==1:
    #break

print(graph2_p[20].info())

print(len(graph2_p))

Streaming output truncated to the last 5000 lines.
42      -2.992345  
43      -0.083905  
44       0.041303  
45      -0.593201  
46      -1.469543  

[5 rows x 256 columns]
StellarGraph: Undirected multigraph
 Nodes: 381, Edges: 3189

 Node types:
  default: [381]
    Features: float32 vector, length 256
    Edge types: default-default->default

 Edge types:
    default-default->default: [3189]
        Weights: all 1 (default)
        Features: float32 vector, length 1
TCGA.05.4249
      0     1  Weight
0   607   561       1
1  1368  1413       1
2  1460  1416       1
3   570   615       1
4   797   843       1
['633.csv']
                1         2         3         4         5         6         7  \
file_id                                                                         
333     -5.302040  1.955478  3.627688 -7.659984 -4.996185 -2.118541 -4.574276   
335     -6.026178 -1.052430  5.067893 -8.080827 -5.807868 -3.361449 -2.553044   
336     -5.004915 -1.088816  0.874583 -3.71

In [12]:
#----Create numpy array of 'zeros' and 'ones' for labelling benign/malignant images resp.
graphp_label1 = np.ones(len(graph_p))
graph2p_label = np.ones(len(graph2_p))
graphp_label = (-1)*graphp_label1

Graph = np.concatenate((graph_p, graph2_p), axis = 0)
label = np.concatenate((graphp_label, graph2p_label), axis = 0)
#X_test = np.concatenate((benign_valid, malign_valid), axis = 0)
#Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

#Y_data = pd.get_dummies(Y_data, drop_first=True)

#Graph = pd.DataFrame(graphd)

labels = pd.DataFrame(label)
print(labels.value_counts().to_frame())

#labels = pd.get_dummies(labels, drop_first=True)
#print(labels)



      count
0          
-1.0    166
 1.0    113


In [13]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt

import pandas as pd
#import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

import numpy as np
np.bool = np.bool_
#!pip install numpy==1.17.0
#import numpy as np

In [14]:
generator = PaddedGraphGenerator(graphs=Graph)

In [15]:
def create_graph_classification_model(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[64, 64],
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=32, activation="relu")(x_out)
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
    #model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"])

    return model



new_model = create_graph_classification_model(generator)
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 256)]          0         []                            
                                                                                                  
 dropout (Dropout)           (None, None, 256)            0         ['input_1[0][0]']             
                                                                                                  
 input_3 (InputLayer)        [(None, None, None)]         0         []                            
                                                                                                  
 graph_convolution (GraphCo  (None, None, 64)             16448     ['dropout[0][0]',             
 nvolution)                                                          'input_3[0][0]']         

In [16]:
epochs = 20  # maximum number of training epochs
folds = 5  # the number of folds for k-fold cross validation
n_repeats = 5  # the number of repeats for repeated k-fold cross validation

learning_rate = 0.005 # 0.01
dropout_rate = 0.5 #0.5
num_epochs = 10
batch_size = 32
comms_round = 100

#model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"])
from tensorflow.keras.optimizers import SGD, RMSprop, AdamW, Adadelta, Adagrad
optimizer=optimizers.legacy.Adam(learning_rate)
        #loss=keras.losses.BinaryCrossentropy(from_logits=True),
loss=losses.BinaryCrossentropy()
metrics =[metrics.SparseCategoricalAccuracy(name="acc")]

In [17]:
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)

In [18]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=0, callbacks=[es],
    )
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen, verbose=0)
    test_acc = test_metrics[model.metrics_names.index("acc")]

    return history, test_acc

In [19]:
def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )

    return train_gen, test_gen

In [23]:
test_accs = []

stratified_folds = model_selection.RepeatedStratifiedKFold(
    n_splits=folds, n_repeats=n_repeats
).split(labels, labels)
new_train = list()
new_test = []
train_index_main = []
test_index_main = []
j = 0

for i, (train_index, test_index) in enumerate(stratified_folds):
    print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
    print(i)
    new_train[j+1] = train_index
    #new_test[j] = test_index
    #train_index_main = np.concatenate(new_train[j])
    #test_index_main = np.concatenate(new_test[j])

    #train_gen, test_gen = get_generators(
     #   train_index, test_index, labels, batch_size=30
    #)
    #j = j+1
'''d
    model = create_graph_classification_model(generator)

    history, acc = train_fold(model, train_gen, test_gen, es, epochs)

    test_accs.append(acc)
'''

Training and evaluating on fold 1 out of 25...
0


IndexError: list assignment index out of range

Federated Members (Clients) as Data Shards

In [22]:
import random
def create_clients(image_list, label_list, num_clients=2, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as
                data shards - tuple of images and label lists.
        args:
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1

    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    print(client_names)
    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

#create clients
clients = create_clients(train_index, labels.iloc[train_index].values, num_clients=2, initial='client')

NameError: name 'train_index' is not defined

Processing and batching clients' and test data

In [ ]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


#process and batch the training data for each client

data = list(zip(train_index, labels.iloc[train_index].values))
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((test_index, labels.iloc[test_index].values)).batch(len(labels.iloc[test_index].values))

Model Aggregation (Federated Averaging)

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

In [ ]:
def test_model(model, comms_round, stratified_folds):
  for i, (train_index, test_index) in enumerate(stratified_folds):
      print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
      train_gen, test_gen = get_generators(
          train_index, test_index, labels, batch_size=30
      )

      model = create_graph_classification_model(generator)

      history, acc = train_fold(model, train_gen, test_gen, es, epochs)

      test_accs.append(acc)

Federated Model Training

In [ ]:
#initialize global model
#smlp_global = SimpleMLP()
#global_model = smlp_global.build(961, 2)
global_model = create_graph_classification_model(generator)

#global_acc_list = []
#global_loss_list = []

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in client_names:
        #smlp_local = SimpleMLP()
        local_model = create_graph_classification_model(generator)
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)
        #model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"])

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(test_index, labels.iloc[test_index].values) in test_batched:
        global_acc, global_loss = test_model(global_model, comm_round, stratified_folds)


In [ ]:
sg.utils.plot_history(history)

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))